In [ ]:
## Styling to center the figures
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# <center>Premier TD de morpho-math</center>

Ce TD parle des motions de connectivité en partant de celle de graphe.

## Connectivité et graphes

Un graphe est un couple $(E,\Gamma)$ où $E$ est un ensemble fini de points, appellés *sommets* et $\Gamma$ une application de $E$ vers ${\cal P}(E)$, la collection des sous-parties de $E$, appellés *arcs*.

## Début: les environnements nécessaires

In [ ]:
## pour avoir des figures "en ligne"
%matplotlib inline

In [ ]:
## le minimum syndical
import matplotlib.pylab as plt
import matplotlib.patches as patches
import numpy as np
from copy import deepcopy

## Une classe de graphe dirigé

In [ ]:
class Digraph(object):
    def __init__(self, nodes, arcs, N, M):
        self.n = N
        self.m = M
        self.nodes = deepcopy(nodes)
        self.coords = self.nodes[:,0:2]
        self.values = self.nodes[:,2]
        self.arcs = deepcopy(arcs)
        self.radius = 0.02
        pass
    
    def get_n(self):
        return self.n
    
    def get_m(self):
        return self.m
    
    def get_nodes(self):
        return self.nodes
    
    def get_arcs(self):
        return self.arcs
    
    ## comment afficher un sommet
    def pdot(self, x,y,col,full=False):
        if full:
            plt.gcf().gca().add_artist(plt.Circle((x,y),self.radius,ec=col,color=col))
        else:
            plt.gcf().gca().add_artist(plt.Circle((x,y),self.radius,ec=col,color='white'))

    ## comment afficher un arc
    def drawArrow(self, A, B):
        plt.arrow(A[0], A[1], B[0] - A[0], B[1] - A[1],
              head_width=.03, length_includes_head=True)
    
    ## the patch idea does not work well
    #style="Simple,tail_width=0.5,head_width=4,head_length=8"
    #kw = dict(arrowstyle=style, color="k")
    def selfArrow(self,A):
        # a = patches.FancyArrowPatch(A, A,connectionstyle="arc3,rad=2", **kw)
        # plt.gcf().gca().add_patch(a)
        center = np.array([0.5,0.5])
        normgrad = (A-center)/np.linalg.norm(A-center)
        B = A+1.2*self.radius*normgrad
        plt.gcf().gca().add_artist(plt.Circle((B[0],B[1]),1.2*self.radius,ec='k',color='white'))

    def print(self):
        ## liste triée par première colonne (i.e. premier élément)
        print("Matrice d'ajacence:\n",self.arcs[self.arcs[:,0].argsort()])

    def show(self):
        plt.figure(figsize=(8,8))
        ax=plt.subplot(aspect='equal')
        plt.axis('off')
        # plots the arcs first
        for j in range(self.m):
            AB=tuple(self.arcs[j])
            #print("Plotting arc between node(%d) and node(%d)" % AB)
            if (AB[0] != AB[1]):
                ## draw arrows a little short
                grad = self.coords[AB[1]]-self.coords[AB[0]]
                normgrad = (grad)/np.linalg.norm(grad)
                self.drawArrow(self.coords[AB[0]], self.coords[AB[1]]-self.radius*normgrad)
            else:
                ## self referencing arrow
                self.selfArrow(self.coords[AB[0]])

        ## plot the nodes
        for i in range(self.n):
            self.pdot(self.coords[i,0], self.coords[i,1],'green',full=self.values[i])
            plt.figtext(self.coords[i,0], self.coords[i,1],"%d"%(i))
        
        plt.show()

## Création d'un graphe aléatoire

In [ ]:
## quelques sommets aléatoires pas trop près du bord
def spread(nbpts):
    x=np.zeros(nbpts) ; y = np.zeros(nbpts)
    for i in range(nbpts):
        x[i] = np.cos(2*i*np.pi/nbpts)/2.3 + 0.5
        y[i] = np.sin(2*i*np.pi/nbpts)/2.3 + 0.5
        
    return np.array((x,y)).T


def createDigraph(N,M):
    coords = spread(N)
    values = np.random.randint(2, size=(N,1))
    arcs = np.random.randint(N, size=(M, 2)) ## arcs from i to j
    return((np.hstack((coords,values)),arcs)) 


In [ ]:
N=16;M=12
(nodes,arcs) = createDigraph(N,M)

G=Digraph(nodes,arcs,N,M)
G.show()
G.print()

## Operateurs sur les graphes dirigés

In [ ]:
def dilation(G):
    nodes=G.get_nodes()
    arcs=G.get_arcs()
    newnodes=deepcopy(nodes)
    for a in arcs:
        ## just the definition
        newnodes[a[1],2]=max(newnodes[a[1],2],nodes[a[0],2])
    return(Digraph(newnodes,arcs,G.get_n(),G.get_m()))

In [ ]:
Gdil=dilation(G)
Gdil.show()

In [ ]:
def nondual_erosion(G):
    """
    This version of the erosion only compute the min instead of the max,
    but does not reverse the graph.
    """
    nodes=G.get_nodes()
    arcs=G.get_arcs()
    newnodes=deepcopy(nodes)
    for a in arcs:
        ## just the definition
        newnodes[a[1],2]=min(newnodes[a[1],2],nodes[a[0],2])
    return(Digraph(newnodes,arcs,G.get_n(),G.get_m()))

### Cette version de l'érosion est-elle duale de la dilatation ?

In [ ]:
Gnondual_ero = nondual_erosion(G)
Gnondual_ero.show()

In [ ]:
Gnotclo=nondual_erosion(dilation(G))
Gnotop.show()

In [ ]:
def isGreater(G1,G2,verbose=True):
    """Check if G1 >= G2, returns a Boolean"""
    retval=True
    n1 = G1.get_nodes()
    n2 = G2.get_nodes()
    for i in range(G1.get_n()):
        if (n1[i,2] < n2[i,2]):
            if (verbose):
                print("test non vérifié pour i =",i)
            retval=False
            break
    return(retval)

In [ ]:
if (isGreater(Gnotclo,G)):
    print("Extensivité vérifié")
else:
    print("Extensivité non vérifié ")

### Conclusion: l'érosion n'est pas duale. Il faut symétriser le graphe d'abord

In [ ]:
def sym(G):
    """Returns a new graph, symmetric graph of the input"""
    nodes=G.get_nodes()
    arcs=G.get_arcs()
    newarcs=deepcopy(arcs)
    for m in range(G.get_m()):
        newarcs[m] = np.flip(arcs[m],0)
    return(Digraph(nodes,newarcs,G.get_n(),G.get_m()))

In [ ]:
G.get_arcs()

In [ ]:
Gsym=sym(G)
Gsym.get_arcs()
Gsym.show()

### Nouvelle version de l'érosion

In [ ]:
def inelegant_erosion(G):
    """
    Erosion of a graph G.
    This is the dual of the dilation, i.e \delta_{\Gamma^{-1}}^\star
    However, the output graph has the same arcs as the input
    """
    Gsym=sym(G) # we compute the symmetric graph
    symarcs=Gsym.get_arcs()
    nodes=G.get_nodes() # same as input
    newnodes=deepcopy(nodes) # because they are going to change
    for a in symarcs: # use the symmetric arcs
        ## just the definition
        newnodes[a[1],2]=min(newnodes[a[1],2],nodes[a[0],2])
    return(Digraph(newnodes,G.get_arcs(),G.get_n(),G.get_m()))

In [ ]:
Gero=inelegant_erosion(G)
Gero.show()

In [ ]:
Gclo=inelegant_erosion(dilation(G))
Gclo.show()

In [ ]:
if (isGreater(Gclo,G)):
    print("Extensivité vérifié")
else:
    print("Extensivité non vérifié ")

### Cette fois-ci, l'extensivité est OK.

Vérifions pour l'ouverture

In [ ]:
def isSmaller(G1,G2,verbose=True):
    """Check if G1 <= G2, returns a Boolean"""
    return(isGreater(G2,G1,verbose))

In [ ]:
Gope=dilation(inelegant_erosion(G))
Gope.show()
if (isSmaller(Gope,G)):
    print("Anti-extensivité vérifié")

### Une version plus élégante de l'érosion

In [ ]:
def erosion(G):
    """
    Erosion of a graph G.
    This is the dual of the dilation, i.e \delta_{\Gamma^{-1}}^\star
    However, the output graph has the same arcs as the input
    """
    arcs=G.get_arcs()
    nodes=G.get_nodes() # same as input
    newnodes=deepcopy(nodes) # because they are going to change
    for a in arcs: # use the symmetric arcs
        ## just the definition
        newnodes[a[0],2]=min(newnodes[a[0],2],nodes[a[1],2]) # we just use the reversed arc
    return(Digraph(newnodes,arcs,G.get_n(),G.get_m()))

In [ ]:
Gope2=dilation(erosion(G))
Gope2.show()
if (isSmaller(Gope2,G)):
    print("Anti-extensivité vérifié")

In [ ]:
%run digraph.py